In [1]:
import pandas as pd
import pyodbc
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [11]:
#fetching data from sql database using a sql query
def fetch_data():
    conn_str=(
        "Driver={SQL Server};"
        "Server=LAPTOP-NQ0K6AQI\\SQLEXPRESS;"
        "Database=PortfolioProject_MarketingAnalytics;"
        "Trusted_Connection=yes;"
    )
    conn=pyodbc.connect(conn_str)
    query="Select ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText from dbo.customer_reviews"
    df=pd.read_sql_query(query,conn)
    conn.close()
    return df
cus_review_df=fetch_data()

C:\Users\Alekhya\AppData\Local\Temp\ipykernel_13484\4058104976.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(query,conn)


In [15]:
nltk.download('vader_lexicon')
sa=SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Alekhya\AppData\Roaming\nltk_data...


In [17]:
#function to calculate sentiment score
def calculate_sentiment(review):
    sentiment=sa.polarity_scores(review)
    return sentiment['compound']

In [19]:
#funtion to categorize sentiment using both score and review rating
def categorize_sentiment(score,rating):
    if score>0.05:
        if rating>=4:
            return "Positive"
        elif rating ==3:
            return "Mixed Positive"
        else:
            return "Mixed Negative"
    elif score<-0.05:
        if rating<=2:
            return "Negative"
        elif rating==3:
            return "Mixed Negative"
        else:
            return "Mixed Positive"
    else:
        if rating>=4:
            return "Positive"
        elif rating<=2:
            return "Negative"
        else:
            return "Neutral"

In [21]:
#function to bucket sentiment scores into ranges
def sentiment_bucket(score):
    if score>=0.5:
        return '0.5 to 1.0'
    elif 0.0<=score<0.5:
        return '0.0 to 0.49'
    elif -0.5<=score<0.0:
        return '-0.49 to 0.0'
    else:
        return '-1.0 to -0.5'

In [23]:
cus_review_df['SentimentScore']=cus_review_df['ReviewText'].apply(calculate_sentiment)

In [27]:
cus_review_df['SentimentCategory']=cus_review_df.apply(
    lambda row:categorize_sentiment(row['SentimentScore'],row['Rating']),axis=1)

In [29]:
cus_review_df['SentimentBucket']=cus_review_df['SentimentScore'].apply(sentiment_bucket)

In [31]:
print(cus_review_df.head())

   ReviewID  CustomerID  ProductID  ReviewDate  Rating  \
0         1          77         18  2023-12-23       3   
1         2          80         19  2024-12-25       5   
2         3          50         13  2025-01-26       4   
3         4          78         15  2025-04-21       3   
4         5          64          2  2023-07-16       3   

                                 ReviewText  SentimentScore SentimentCategory  \
0   Average  experience,  nothing  special.         -0.3089    Mixed Negative   
1            The  quality  is    top-notch.          0.0000          Positive   
2   Five  stars  for  the  quick  delivery.          0.0000          Positive   
3  Good  quality,  but  could  be  cheaper.          0.2382    Mixed Positive   
4   Average  experience,  nothing  special.         -0.3089    Mixed Negative   

  SentimentBucket  
0    -0.49 to 0.0  
1     0.0 to 0.49  
2     0.0 to 0.49  
3     0.0 to 0.49  
4    -0.49 to 0.0  


In [37]:
cus_review_df=pd.DataFrame(cus_review_df)

In [39]:
print(cus_review_df.head())

   ReviewID  CustomerID  ProductID  ReviewDate  Rating  \
0         1          77         18  2023-12-23       3   
1         2          80         19  2024-12-25       5   
2         3          50         13  2025-01-26       4   
3         4          78         15  2025-04-21       3   
4         5          64          2  2023-07-16       3   

                                 ReviewText  SentimentScore SentimentCategory  \
0   Average  experience,  nothing  special.         -0.3089    Mixed Negative   
1            The  quality  is    top-notch.          0.0000          Positive   
2   Five  stars  for  the  quick  delivery.          0.0000          Positive   
3  Good  quality,  but  could  be  cheaper.          0.2382    Mixed Positive   
4   Average  experience,  nothing  special.         -0.3089    Mixed Negative   

  SentimentBucket  
0    -0.49 to 0.0  
1     0.0 to 0.49  
2     0.0 to 0.49  
3     0.0 to 0.49  
4    -0.49 to 0.0  


In [41]:
cus_review_df.to_csv('fact_customer_reviews_with_sentiment.csv', index=False)